# Assignment 5 - Neural Architecture Search (NAS)

In this assignment, Neural Architecture Search (NAS) will be implemented. The goal of NAS is to find the best neural network architecture for a given task. The dataset that will be used in this assignment is SVHN (same as assignment 4). The possible configurations of the NN are given in the instructions. 


In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import datasets
from sklearn.datasets import load_digits
from torch.utils.data import DataLoader, Dataset

EPS = 1.0e-7

In [ ]:
# Colab settings
from google.colab import drive

drive.mount("/content/gdrive")
results_dir = "/content/gdrive/MyDrive/CI"


## Dataset

The same dataset code from assignment 4 will be used here. The dataset is already downloaded in the `data` folder. The dataset is loaded in the code below. Moreover, the neccessary code for carrying dataloaders to GPU is given below.

In [ ]:
from torchvision.datasets import SVHN
from torch.utils.data import Dataset, random_split
from torchvision import transforms
from sklearn.model_selection import train_test_split

# svhn dataset from torchvision is used for effective data download
SVHN(root =  "/content/gdrive/MyDrive/CI" ,split='test', download=True)
SVHN(root =  "/content/gdrive/MyDrive/CI" ,split='train', download=True)


In [ ]:

class HouseNumbers(Dataset):
    """House Numbers Dataset."""

    def __init__(self, root = './', mode="train", transform=None, val_ratio=0.2):
        """
        Dataset Class for House Numbers Dataset. 
        It uses SVHN dataset from torchvision to download the dataset but only uses the train and test splits.
        It creates its own data and target variables by splitting the train set into train and validation sets. 
        So that it can be used in a standard way for training, validation, and test purposes with PyTorch DataLoader.
        Args:
            root (string): Directory with all the images (default: './')
            mode (string): train, val, or test (default: train)
            transforms (function): Optional transform to be applied on a sample.
            val_ratio (float): Ratio of validation set when mode is train (default: 0.2)
        """
        self.transform = transform
        self.mode = mode
        self.val_ratio = val_ratio
        self.root = root

        if self.mode == 'test':
            # load test set
            test_set = SVHN(root=self.root, split='test', transform=transforms.ToTensor())
            # set data and target variables
            self.data = test_set.data
            self.target = test_set.labels
        else:
            # decide train and validation indices 
            # set random seed to 0 for achieving the same result in each instance of the class
            train_indices, val_indices = train_test_split(
                                            np.arange(len(SVHN(root=self.root, split='train'))), 
                                            test_size=self.val_ratio, 
                                            random_state=0)
            # if mode is train, load train set
            if self.mode == 'train':
                complete = SVHN(root=self.root, split='train', transform=transforms.ToTensor())
                self.data = torch.utils.data.Subset(complete.data, train_indices)
                self.target = np.array(complete.labels)[train_indices]
            # if mode is val, load validation set
            elif self.mode == 'val':
                complete = SVHN(root=self.root, split='train', transform=transforms.ToTensor())
                self.data = torch.utils.data.Subset(complete.data, val_indices)
                self.target = np.array(complete.labels)[val_indices]
            else :
                raise ValueError('Invalid mode %s' % self.mode)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample_x = self.data[idx]
        sample_y = self.target[idx]
        if self.transform:
            sample_x = self.transform(sample_x)
        return (sample_x, sample_y)        

In [ ]:
# Initialize training, validation and test sets.
train_data = HouseNumbers(mode="train", root= "/content/gdrive/MyDrive/CI")
val_data = HouseNumbers(mode="val",  root= "/content/gdrive/MyDrive/CI")
test_data = HouseNumbers(mode="test",  root= "/content/gdrive/MyDrive/CI")

# Initialize data loaders.
training_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
# move dataloaders to device as well
# otherwise the training process will give an error
# model parameters will be on the device and have type torch.cuda.FloatTensor
# but the data will be on the CPU and have type torch.FloatTensor 
# so we need to move the data to the device that the model is on to make sure they are the same type
# to fix this issue, found following solution in the pytorch documentation 
# modified it to fit our needs
# ref: https://pytorch.org/tutorials/beginner/nn_tutorial.html#wrapping-dataloader
class WrappedDataLoaderCustomize():
    def __init__(self, dl, device):
        self.dl = dl
        # in the original code, there is a function 
        # but here it will be fixed: 'to_device' so not included
        self.device = device

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        for i in self.dl:
            yield self.move_to_device(i, self.device)
    
    def move_to_device(self, b, device):
        # since the our data can be tuple shaped, we need to move each element to the device recursively
        # check the input type
        # move each element to the device recursively
        if isinstance(b, (list, tuple)):
            return [self.move_to_device(x, device) for x in b]
        # move the input to the device directly
        return b.to(device)
      

## Neural Network Architecture

This section includes the implementation of a convolutional neural network of the following form:

    Conv2d → f(.) → Pooling → Flatten → Linear 1 → f(.) → Linear 2 → Softmax 
    
However, different choices in each building block are allowed as follows:

● Conv2d:
  - Number of filters: 8, 16, 32
  - kernel=(3,3), stride=1, padding=1 OR kernel=(5,5), stride=1, padding=2
  
● f(.):
  - ReLU OR sigmoid OR tanh OR softplus OR ELU

● Pooling:
  - 2x2 OR Identity
  - Average OR Maximum

● Linear 1:
  - Number of neurons: 10, 20, 30, 40, 50, 60, 70, 80, 90, 100

Altogether, there are 4500 possible configurations.